# Cryosphere model Comparison tool (CmCt) --- Gravimetry

The CmCt GRACE mascon tool compares user uploaded ice sheet model to satellite gravimetry in the form of mascon data, derived from the GRACE and GRACE-FO satellite missions. The CmCt reprojects modeled ice sheet mass change into mascon space and performs a direct quantitative comparison, calculating model-observation discrepancies, therefore bringing the ice sheet model and gravimetry data to the same spatial resolution for comparison. 

## Mascon dataset description

For comparison to the user's model, the tool uses the NASA GSFC GRACE mascon product, which can be downloaded from [here](https://earth.gsfc.nasa.gov/geo/data/grace-mascons/). Future developments will include adding the functionality to automatically check the product distribution website for an updated dataset.

## Input data requirements

The input ice sheet model needs to be provided as a netCDF file. The user may upload a single input file that includes multiple years; future enhancements to this tool will allow users to upload model ensembles.

There are several requirements for the comparison:

### `Lithk` variable

The CmCt Grace Mascon tool expects the uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](https://theghub.org/groups/ismip6/wiki). 

### Date range

The NASA GSFC GRACE mascon dataset starts on 04/2002 and is updated ~quarterly. The user can select start and end dates within this span as part of the setup for the tool.

## Tool output

from gravimetry_utils import *This tool creates a plot that GRACE mascon plot and the model mascon plot so the user can  qualitatively compare the model output with the observational data. 

In [ ]:
## Import modules
import os,sys
import numpy as np
import cftime 

# Add the directory containing 'cmct' to the Python path
# Navigate two levels up to reach main CmCt dir
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Import utilities for this comparison
sys.path.insert(0,cmct_dir)
from cmct.time_utils import check_datarange
from cmct.mascons import *
from cmct.gravimetry import *
from cmct.projection import *


## Configure gravimetry comparison

In [ ]:
# Ice sheet
loc = 'GIS' # 'GIS' or 'AIS'

# Set time range for comparison
start_date = '2006-01-01' 
end_date = '2014-12-31' 

# Ice thickness (m) to cm water equivalent:
rho_ice = 918 # kg/m^3
rho_water = 1000 # kg/m^3

# Set the observation data dir path
obs_filename = cmct_dir + '/data/Gravimetry/gsfc.glb_.200204_202410_rl06v2.0_obp-ice6gd.h5'

# Set the shape data dir path
shape_filename = cmct_dir + '/data/ne_10m_coastline/ne_10m_coastline.shp'

# Set the model data dir path
if loc == "GIS":
    # Greenland
    nc_filename = cmct_dir + '/test/gris_dmdt_filt.nc'
elif loc == "AIS":    
    # Antartica
    nc_filename = cmct_dir + '/test/ais_dmdt_grounded_filt.nc'

# Set the output plot file names
plot_filename = os.path.splitext(os.path.basename(nc_filename))[0] + '_mascon_comp'
output_plot_filename = cmct_dir + '/notebooks/Gravimetry/'+ plot_filename + '.png'
output_netcdf_filename = cmct_dir + '/notebooks/Gravimetry/'+ plot_filename + '.nc'


In [ ]:
# Check if observation file exist
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# Check if model file exist    
if not os.path.exists(nc_filename):
    raise FileNotFoundError(f"Model file not found: {nc_filename}")
    

## Set polar sterographic projection definition
Currently, the CmCt expects the model data to be provided in a specific polar stereographic projection. This is hardcoded in the notebook. As a future enhancement, the projection can be obtained from the loaded model.

In [ ]:
polar_stereographic = set_projection(loc)

## Process observation

In [ ]:
# Load observation data(mascons)
gsfc = loadGsfcMascons(obs_filename)

# Compute the mascon means and calulate mass change of observation data
mass_change_obs,I_ = computeMasconMeans(gsfc, start_date, end_date, loc)


## Process ice sheet model

In [ ]:
# Load model data
gis_ds = loadGisModel(nc_filename)
time_var = gis_ds['time']

# Convert start/end comparison times to fractional year
calendar_type = time_var.to_index().calendar
start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Adjust day to be 30 ( to avoid error if it's the 31st day in a 360_day calendar)
start_date_cftime = cftime.datetime(start_date_dt.year, start_date_dt.month, min(start_date_dt.day, 30), calendar=calendar_type)
end_date_cftime = cftime.datetime(end_date_dt.year, end_date_dt.month, min(end_date_dt.day, 30), calendar=calendar_type)

# Check the selcted dates are within the range of model data
check_datarange(time_var,start_date_cftime, end_date_cftime)

# Put model into mascon space and calulate mass change of model data
mass_change_mod_trim, mass_change_mod = transformToGeodetic(gsfc, gis_ds, start_date_cftime, end_date_cftime, rho_ice, rho_water, polar_stereographic)


### Check the selcted dates are within the range of model data

In [ ]:
# Calculate mass change of model and observation data
mass_change_delta = mass_change_mod_trim-mass_change_obs

# Write result to nc file
write_to_netcdf(mass_change_obs, mass_change_delta, mass_change_mod_trim, gsfc, I_, start_date_cftime, end_date_cftime, output_netcdf_filename)

# Plot results
plotFigure(mass_change_obs, mass_change_mod_trim, mass_change_delta, gsfc, I_, start_date_cftime, end_date_cftime, polar_stereographic, loc, shape_filename, output_plot_filename)
